## Enhance under water images with U-shape transformer without resizing image! 👀

### [U-shape Transformer Underwater Image Enhancement]

paper: https://arxiv.org/abs/2111.11843

code (pytorch): https://github.com/LintaoPeng/U-shape_Transformer

Before I start with U-shape Transformer, I found out that this code resize the output image into 256x256 which is not efficient when training with detection code.

So I edit some code lines to handle images by croping into 256 x 256 patches and preprocess each piece.  

I refer to code provided in korean competition site "DACON" which I linked below!

reference: https://dacon.io/competitions/official/235746/codeshare/2874?page=2&dtype=recent

In [ ]:
%cd /kaggle/working/
!git clone https://github.com/LintaoPeng/U-shape_Transformer_for_Underwater_Image_Enhancement.git
%cd /kaggle/working/U-shape_Transformer_for_Underwater_Image_Enhancement

In [ ]:
!pip install -r requirements.txt

#### download saved_model.zip folder where models saved and place it where you want

https://drive.google.com/file/d/19a_kDJTT5S96kzwQntEMhSxAPYw4xY2P/view

In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import time
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm
from glob import glob
from torch.nn.modules.loss import _Loss
from net.Ushape_Trans import *
#from dataset import prepare_data, Dataset
from net.utils import *
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *
from torchvision.utils import save_image

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
dtype = 'float32'

torch.set_default_tensor_type(torch.FloatTensor)


def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

# Initialize generator
generator = Generator().cuda()
generator.load_state_dict(torch.load("/kaggle/input/models/saved_models/G/generator_795.pth"))


img_size = 256
path = '/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0' # data directory



def predict(img_paths, stride=32, batch_size=1):
    results = []
    for img_path in os.listdir(img_paths):
        img = cv2.imread(os.path.join(img_paths, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img = np.array(img).astype(dtype)
        #img = torch.from_numpy(img)  # tensor (h, w, c)


        crop = []
        position = []
        batch_count = 0

        result_img = np.zeros_like(img)
        voting_mask = np.zeros_like(img)
        i = 0
        for top in tqdm(range(0, img.shape[0], stride)):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([img_size, img_size, 3], np.float32)
                temp = img[top:top + img_size, left:left + img_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                crop.append(piece)
                position.append([top, left])
                batch_count += 1
                if batch_count == batch_size:


                    crop = np.array(crop).astype(dtype)
                    crop = torch.from_numpy(crop)  #tensor (b, h, w, c)
                    crop = crop.permute(0, 3, 1, 2) # tensor (b, c, h, w)
                    #.unsqueeze(0) => make batch 1
                    crop = crop / 255.0
                    crop = Variable(crop).cuda()
                    output = generator(crop)*255
                    pred = output[3].data

                    # check cropped image output
                    save_image(pred, "./test/output/cropped_" + img_path, nrow=5, normalize=True)

                    crop = []
                    batch_count = 0
                    for num, (t, l) in enumerate(position):
                        piece = pred[num]
                        piece = piece.permute(1, 2, 0)
                        h, w, c = result_img[t:t + img_size, l:l + img_size, :].shape
                        #result_img = torch.Tensor(result_img).cuda()
                        piece = piece.cpu().detach().numpy()
                        result_img = result_img.astype(dtype)
                        result_img[t:t + img_size, l:l + img_size, :] += piece[:h, :w, :]
                        voting_mask[t:t + img_size, l:l + img_size, :] += 1
                    position = []


        result_img = result_img / voting_mask
        #result_img = result_img.astype(np.uint8)
        #results.append(result_img)
        results = np.array(result_img).astype(dtype)
        results = torch.from_numpy(results)
        results = results.permute(2, 0, 1).unsqueeze(0)# tensor (b, c, h, w)
        # save image to your output folder dir
        save_image(results, "../outputdir/" + img_path, nrow=5, normalize=True)

    return results


if __name__ == "__main__":

    predict(path)